In [1]:
import pandas as pd
import numpy as np
from transformers import pipeline
import pycountry
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import pandas as pd
from tqdm import tqdm
import pycountry
import logging
import country_converter as coco

In [3]:
df=pd.read_csv(r'C:\Users\Yasaman\Arab_spring_scholarly_attention\Validating\final_annotated.csv')
df['Text']=df['Title']+' '+df['Abstract']
df['union_annotation'] = df['union_annotation'].apply(lambda x: eval(x) if isinstance(x, str) else x)
df['intersection_annotation'] = df['intersection_annotation'].apply(lambda x: eval(x) if isinstance(x, str) else x)

In [4]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
country_names = {country.name for country in pycountry.countries}

# Function to extract locations and filter by country names
def extract_and_filter_locations(text):
    ner_results = nlp(text)
    locations = [
        entity["word"]
        for entity in ner_results
        if ( entity["entity"] == "B-LOC" or entity["entity"] == "I-LOC") 
    ]

    final=coco.convert(list(set(locations)), to='ISO3') 
    if type(final)==str:
        final=[final]

    final=[x.lower() for x in final if x!='not found']
   
    return final

# Apply the function to the DataFrame with tqdm
tqdm.pandas()  # Initialize tqdm for pandas

coco_logger = coco.logging.getLogger()
coco_logger.setLevel(logging.CRITICAL)
df["locations"] = df["Text"].progress_apply(extract_and_filter_locations)

# Display the result


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 1000/1000 [11:17<00:00,  1.47it/s]


In [7]:
df.to_csv(r'C:\Users\Yasaman\Arab_spring_scholarly_attention\Validating\bert_base_annotated.csv', index=False)

In [8]:
print('overall accuracy (union)', sum(df['locations']==df['union_annotation'])/1000)
print('overall accuracy (intersection)', sum(df['locations']==df['intersection_annotation'])/1000)

overall accuracy (union) 0.724
overall accuracy (intersection) 0.71


In [9]:
sample_accuracies = df.groupby("SampleGroup").apply(
    lambda x: pd.Series({
        "accuracy_union": (x.apply(lambda row: set(row['locations']) == set(row['union_annotation']), axis=1).mean()),
        "accuracy_intersection": (x.apply(lambda row: set(row['locations']) == set(row['intersection_annotation']), axis=1).mean())
    })
).reset_index()

sample_accuracies = sample_accuracies.sort_values(by='accuracy_union').reset_index(drop=True)
sample_accuracies

C:\Users\Yasaman\AppData\Local\Temp\ipykernel_28204\2343847047.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_accuracies = df.groupby("SampleGroup").apply(


,SampleGroup,accuracy_union,accuracy_intersection
0,with_mention_arab,0.605000,0.570000
1,with_mention,0.632143,0.582143
2,field_20,0.896154,0.894231


In [10]:
sample_accuracies = df.groupby("SampleGroup").apply(
    lambda x: pd.Series({
        "jaccard_union": (
            x.apply(
                lambda row: len(set(row['locations']).intersection(set(row['union_annotation']))) /
                            len(set(row['locations']).union(set(row['union_annotation'])))
                if len(set(row['locations']).union(set(row['union_annotation']))) > 0 else 1,
                axis=1
            ).mean()
        ),
        "jaccard_intersection": (
            x.apply(
                lambda row: len(set(row['locations']).intersection(set(row['intersection_annotation']))) /
                            len(set(row['locations']).union(set(row['intersection_annotation'])))
                if len(set(row['locations']).union(set(row['intersection_annotation']))) > 0 else 1,
                axis=1
            ).mean()
        )
    })
).reset_index()

sample_accuracies = sample_accuracies.sort_values(by='jaccard_union').reset_index(drop=True)
sample_accuracies

C:\Users\Yasaman\AppData\Local\Temp\ipykernel_28204\388812476.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_accuracies = df.groupby("SampleGroup").apply(


,SampleGroup,jaccard_union,jaccard_intersection
0,with_mention,0.715000,0.668810
1,with_mention_arab,0.795681,0.735680
2,field_20,0.908045,0.904519
